In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tfs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D,LeakyReLU
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2, random

In [ ]:
height=28
width=28
depth=1

inputShape = (height, width, depth)

In [ ]:
# Prepare the train and test dataset.
from tensorflow import keras
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


In [ ]:
from  keras.utils import np_utils
# y_train = np_utils.to_categorical(y_train)
# y_test= np_utils.to_categorical(y_test)
print(y_train[0])
print(y_train.shape,y_test.shape)
print(x_train.shape)
print(y_train[0],x_train[0])

5
(60000,) (10000,)
(60000, 28, 28, 1)
5 [[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.     

In [ ]:
from sklearn.utils import shuffle
x_train, y_train= shuffle(x_train, y_train, random_state=0)
X=[]
Y=[]
j=0
for i in range(20):
  X.append(x_train[j:j+3000])
  Y.append(y_train[j:j+3000])
  j+=3000



In [ ]:
X=np.array(X)
Y=np.array(Y)
print(X.shape,Y.shape)

(20, 3000, 28, 28, 1) (20, 3000)


In [ ]:
from tensorflow.keras.layers import BatchNormalization
class MODEL:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Conv2D(128,(3,3),strides=(2, 2), padding="same",input_shape=inputShape) )
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
        model.add(Dropout(0.25))


        model.add(Conv2D(256, (3, 3), strides=(2, 2), padding="same") )
        model.add(Flatten())
        model.add(Dense(128,activation='relu'))
        model.add(Dense(10))#for output layer
        return model

In [ ]:
from sklearn.metrics import accuracy_score
def test_model(X_test, Y_test,  model, comm_round):
    # cce = tf.keras.losses.SparseCategoricalCrossentropy()
    # #logits = model.predict(X_test, batch_size=100)
    # logits = model.predict(X_test)
    # loss = cce(Y_test, logits)
    # acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    # print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    model.compile(  optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
    # loss,acc=model.evaluate(x_test,y_test)
    acc,loss=model.evaluate(x_test,y_test)
    return acc, loss
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
      
    return avg_grad

In [ ]:
from tensorflow import keras

In [ ]:
global1= MODEL()
global_model = global1.build()
comms_round = 50
# print(trainy[2].shape)
f= open("FLbasicMNIST.txt", "a+") 
f.close()
epoch=np.random.randint(1,6,20)
print(epoch)
from tensorflow.keras import backend as K
for comm_round in range(150):
  f= open("FLbasicMNIST.txt", "a+") 
  global_weights = global_model.get_weights()
  scaled_local_weight_list = list()
  index=list({0,1,2,3,4})
  #random.shuffle(index)
  # print(index)
  f.write("\nCommunication Round:%d \n" %comm_round)
  print("Communication Round:",comm_round)
  for ind in range(20):
    print("CLIENT NO: ",ind)
    f.write("Client No: %d\n" %ind)
    local = MODEL()
    local_model=local.build()
    
    local_model.compile(optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
    local_model.set_weights(global_weights)
    #trainy[ind]=np.array(trainy[ind]).reshape(-1,1)
    history=local_model.fit(X[ind],Y[ind], epochs=epoch[ind])#,validation_data=(x_test,y_test))
    # print("Accuracy: ",history.history["accuracy"][4])
    # local_model.evaluate(x_test, y_test)
    local_loss, local_acc = test_model(x_test,y_test, local_model, comm_round)
    f.write("Communication Round: %d Local Model ACCURACY: %f LOSS: %f \n" %(comm_round ,local_acc ,local_loss))
    scaling_factor=1.0/20 #1/no.ofclients
    scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
    scaled_local_weight_list.append(scaled_weights)
    K.clear_session()

  average_weights = sum_scaled_weights(scaled_local_weight_list)
  #global_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  global_model.set_weights(average_weights)
  #val_acc,val_loss=global_model.evaluate(x_train,y_train)
  #print(val_loss,val_acc)
  global_loss, global_acc = test_model(x_test,y_test, global_model, comm_round)
  f.write("\nCommunication Round: %d GLOBAL MODEL ACCURACY: %f LOSS: %f \n" %(comm_round ,global_acc ,global_loss))

        


  


  f.close()

Streaming output truncated to the last 5000 lines.
Epoch 1/2
94/94 [==============================] - 2s 11ms/step - loss: 0.0017 - sparse_categorical_accuracy: 0.9997
Epoch 2/2
313/313 [==============================] - 2s 6ms/step - loss: 0.1668 - sparse_categorical_accuracy: 0.9903
CLIENT NO:  4
313/313 [==============================] - 2s 5ms/step - loss: 0.2138 - sparse_categorical_accuracy: 0.9881
CLIENT NO:  5
Epoch 1/5
94/94 [==============================] - 2s 11ms/step - loss: 4.3870e-07 - sparse_categorical_accuracy: 1.0000
Epoch 2/5
94/94 [==============================] - 1s 11ms/step - loss: 2.9876e-04 - sparse_categorical_accuracy: 0.9997
Epoch 3/5
94/94 [==============================] - 1s 11ms/step - loss: 0.0037 - sparse_categorical_accuracy: 0.9993
Epoch 4/5
94/94 [==============================] - 1s 11ms/step - loss: 0.1033 - sparse_categorical_accuracy: 0.9907
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.4614 - sparse_categorical_a

In [ ]:
print(global_loss)